In [17]:
!pip install pandas
!pip install reportlab


[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python3 -m pip install --upgrade pip
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 28.6 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.4/199.4 kB 39.1 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python3 -m pip install --upgrade pip


In [3]:
import pandas as pd

file_path = r"Article_data.csv"
df = pd.read_csv(file_path);

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 536750 entries, 0 to 536749
Data columns (total 8 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   article_id      536750 non-null  object
 1   newspaper_name  536750 non-null  object
 2   edition         536750 non-null  int64 
 3   date            536750 non-null  object
 4   page            536750 non-null  object
 5   headline        327755 non-null  object
 6   byline          8766 non-null    object
 7   article         536750 non-null  object
dtypes: int64(1), object(7)
memory usage: 32.8+ MB


In [20]:
df.head()

,newspaper_name,date,page,article
0,The Florida agriculturist.,1889-06-05,p13,general horace porter says that when he was in...
1,The Florida agriculturist.,1889-06-05,p13,news notes and comments from the fountain city...
2,The Florida agriculturist.,1889-06-05,p13,a prisoner in the sanford jail muir ders a dep...
3,The Florida agriculturist.,1889-06-05,p13,did at it's june meeting. the county school bo...
4,The Florida agriculturist.,1889-06-05,p13,were misses boacn ana falm0t anu messrs. colby...


In [5]:
df.drop(columns=['byline', 'edition','headline'], inplace=True)

In [6]:
df.describe()

,article_id,newspaper_name,date,page,article
count,536750,536750,536750,536750,536750
unique,536750,165,365,123,531104
top,42_1889-02-23_p4_sn84025834_00220720366_188902...,Los Angeles daily herald.,1889-12-21,p2,Children Cry for Pitchers Castoria.
freq,1,65640,3127,99303,64


In [11]:

import re

def clean_text(text):
    # Remove all characters except letters, numbers, and specified punctuation
    text = re.sub(r'[^a-zA-Z0-9,.!?;:()\'\"]', ' ', text)

    # Replace newline, carriage return, and tab characters with a space
    text = re.sub(r'[\n\r\t]', ' ', text)

    # Replace multiple spaces with a single space
    text = re.sub(' +', ' ', text)

    # Convert text to lower case
    text = text.lower()

    # Normalize white space
    text = re.sub(r'\s+', ' ', text)

    # Replace non-standard punctuation with standard ones
    text = re.sub(r'[“”]', '"', text)
    text = re.sub(r"[‘’]", "'", text)

    # Remove excessive punctuation
    text = re.sub(r'\.{2,}', '.', text)

    # Remove any remaining non-ASCII characters (optional)
    text = re.sub(r'[^\x00-\x7F]+', '', text)

    # Sentence boundary correction
    text = re.sub(r'(?<=[.!?])\s+(?=[A-Z])', ' ', text)

    return text.strip()

# Assuming df is your DataFrame containing an 'article' column
df['article'] = df['article'].apply(clean_text)

# Filter the DataFrame to only include rows where the cleaned article has 200 or more characters
df = df[df['article'].str.len() >= 200]


In [13]:
df.drop(columns=['cleaned_article', 'article_id'], inplace=True)

In [14]:
import pandas as pd
import numpy as np

# Assuming 'data' is your DataFrame
def chunk_dataframe(data, chunk_size):
    num_chunks = len(data) // chunk_size + (1 if len(data) % chunk_size else 0)
    return np.array_split(data, num_chunks)

# Usage
chunk_size = 100000
chunks = chunk_dataframe(df, chunk_size)


/usr/local/lib/python3.11/dist-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


In [18]:
from reportlab.lib.pagesizes import letter
from reportlab.pdfgen import canvas

def create_pdf(chunk, chunk_index):
    filename = f"articles_chunk_{chunk_index}.pdf"
    c = canvas.Canvas(filename, pagesize=letter)
    width, height = letter
    
    y_position = height - 50  # Start 40 pixels down from the top
    line_height = 14  # Height of each line

    for index, row in chunk.iterrows():
        for column in chunk.columns:
            text = f"{column}: {row[column]}"
            c.drawString(40, y_position, text)
            y_position -= line_height  # Move down for the next line
        
        # Add a space between articles
        y_position -= 10

        # Check if we're near the bottom of the page
        if y_position < 40:
            c.showPage()
            y_position = height - 50  # Reset y position

    c.save()

# Example usage
for i, chunk in enumerate(chunks):
    create_pdf(chunk, i)
